In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from scipy import stats
import astropy.units as u
import astropy.constants as c

WDIR = os.environ['DM21CM_DIR']
sys.path.append(WDIR)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file(f"{WDIR}/matplotlibrc")

## 1. PBH

In [2]:
mass_s, inj_s, sigma_s = np.loadtxt(f"{WDIR}/outputs/limits/pbh_limits.txt", unpack=True)

In [10]:
plot_lim_s = [s.split('.txt')[0] for s in os.listdir(f"{WDIR}/data/limits/pbh") if s not in ['EDGESevap-Mittal2021', 'Evaporation.txt']]
print(plot_lim_s)

['EGRB', 'LeoTevap', 'SuperK', 'Voyager', 'EDGESevap-Mittal2021', 'Comptel', 'CMBevap', 'INTEGRAL', '511keV']


In [27]:
lim = {}
for lim_name in plot_lim_s:
    m, f = np.loadtxt(f"{WDIR}/data/limits/pbh/{lim_name}.txt", unpack=True) # [Msun], [1]
    m_g = m * (c.M_sun.to(u.g)).value # [g], [1]
    lim[lim_name] = dict(m=m_g, f=f)

colors = {
    'CMBevap' : cms['binary'](0.6),
    #'Voyager' : cms['binary'](0.4),
    'EGRB' : cms['binary'](0.5),
    'LeoTevap' : cms['binary'](0.35),
    'Comptel' : cms['binary'](0.2),
    'EDGESevap-Mittal2021' : cms['binary'](0.7),
}
zorders = {
    'CMBevap' : -1,
    'Voyager' : -2,
    'EGRB' : -3,
    'LeoTevap' : -4,
    'Comptel' : -5,
    'EDGESevap-Mittal2021' : -6,
}

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

one_sigma = inj_s * sigma_s
upper_limit = np.sqrt(stats.chi2.ppf(.9, df=1)) * one_sigma

f_max = 1e10
for i, n in enumerate(['CMBevap', 'EGRB', 'LeoTevap', 'Comptel']):
    ax.fill_between(lim[n]['m'], lim[n]['f'], f_max, label=n, color=colors[n], zorder=zorders[n])
    #ax.plot(lim[n]['m'], lim[n]['f'], label=n, color=f'C{i}')
n = 'EDGESevap-Mittal2021'
ax.plot(lim[n]['m'], lim[n]['f'], color=cms['binary'](0.8), lw=1, ls='--', zorder=5, label=n)

ax.plot(mass_s, upper_limit, 'k-s', lw=1, ms=2)
ax.fill_between(mass_s, upper_limit+one_sigma, upper_limit-one_sigma, color=mpl.colormaps['viridis'](0.75), alpha=0.8, ec='none')
ax.fill_between(mass_s, upper_limit+2*one_sigma, upper_limit+one_sigma, color=mpl.colormaps['viridis'](1.0), alpha=0.8, ec='none')

ax.set(xscale='log', yscale='log')
ax.set(xlabel=r'$M_\mathrm{PBH}$ [g]', ylabel=r'$f_\mathrm{PBH}=\Omega_\mathrm{PBH}/\Omega_\mathrm{DM}$')
ax.set(ylim=(1e-10, 1e0), xlim=(1e15, 1e18))
ax.legend(loc='lower right', frameon=False)

fig.savefig('pbh_limits.pdf', bbox_inches='tight', dpi=300)